#STREAMING

In [0]:
dbutils.fs.mkdirs("FileStore/tables/stream/landing_files/")
dbutils.fs.mkdirs("FileStore/tables/stream/output_files/")
dbutils.fs.mkdirs("FileStore/tables/stream/stream_checkpoints/")

Out[6]: True

In [0]:
dbutils.fs.ls("dbfs:/FileStore/tables/stream/landing_files/")

Out[22]: []

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

resume_sales_schema = StructType([
    StructField("file",StringType(),True),
    StructField("vendor",StringType(),True),
    StructField("profit",IntegerType(),True)
])

##Read streaming data aggregated

In [0]:
df_resume_sales = spark.readStream.format("csv").schema(resume_sales_schema).option("header",True).load("dbfs:/FileStore/tables/stream/landing_files/")

display(df_resume_sales.groupBy("vendor").sum("profit"))

vendor,sum(profit)
grigor,728
mile,1078
paulo,1587
zack,973
andres,867


##Write streaming data

In [0]:
dbutils.fs.rm("dbfs:/FileStore/tables/stream/landing_files/",True)

Out[18]: True

In [0]:
dbutils.fs.mkdirs("FileStore/tables/stream/landing_files/")

Out[20]: True

In [0]:
dbutils.fs.ls("dbfs:/FileStore/tables/stream/landing_files/")

Out[21]: []

In [0]:
df_read_resume_sales = spark.readStream.format("csv").schema(resume_sales_schema).option("header",True).load("dbfs:/FileStore/tables/stream/landing_files/")

In [0]:
df_write_resume_sales = df_read_resume_sales.writeStream.format("parquet").outputMode("append").option("path","dbfs:/FileStore/tables/stream/output_files/").option("checkpointLocation","dbfs:/FileStore/tables/stream/stream_checkpoints/").start().awaitTermination()

In [0]:
dbutils.fs.ls("dbfs:/FileStore/tables/stream/output_files/")

Out[29]: [FileInfo(path='dbfs:/FileStore/tables/stream/output_files/_spark_metadata/', name='_spark_metadata/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/stream/output_files/part-00000-3dd7dc3f-17a4-48f7-91a3-6c96adfc1fca-c000.snappy.parquet', name='part-00000-3dd7dc3f-17a4-48f7-91a3-6c96adfc1fca-c000.snappy.parquet', size=1171, modificationTime=1723509450000),
 FileInfo(path='dbfs:/FileStore/tables/stream/output_files/part-00000-ae62440d-ccf3-4b5c-aa26-fdbce895e661-c000.snappy.parquet', name='part-00000-ae62440d-ccf3-4b5c-aa26-fdbce895e661-c000.snappy.parquet', size=1167, modificationTime=1723509440000)]

In [0]:
display(spark.read.format("parquet").load("dbfs:/FileStore/tables/stream/output_files/*.parquet"))

file,vendor,profit
resume_2,andres,215
resume_2,paulo,246
resume_2,grigor,452
resume_2,mile,12
resume_2,zack,45
resume_1,andres,10
resume_1,paulo,1000
resume_1,grigor,25
resume_1,mile,80
resume_1,zack,null
